<a href="https://colab.research.google.com/github/empyreanlee/Administrator/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('shampoo_products_data.csv')
df.head()

In [ ]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def get_embeddings(text, model):
  inputs = tokenizer(text, return_tensors="pt", truncation=True, padding= True)
  with torch.no_grad():
    outputs=model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

embeddings = [get_embeddings(text, model=model) for text in df['Product_Details'].values]

In [ ]:

#embeddings_array = np.vstack(embeddings)
#pca = PCA(n_components=2)
#reduced_embeddings = pca.fit_transform(embeddings_array)
reduced_embeddings = embeddings

similarity_matrix = np.zeros((len(reduced_embeddings), len(reduced_embeddings)))
for i in range(len(reduced_embeddings)):
  for j in range(len(reduced_embeddings)):
    if i == j:
      similarity_matrix[i][j] =1.0
    else:
      similarity_matrix[i][j] = cosine_similarity(reduced_embeddings[i].reshape(1,-1), reduced_embeddings[j].reshape(1,-1))[0][0]
    #else:
     # similarity_matrix[i][j] = similarity_matrix[i][j]

In [ ]:
p_name = pd.Series(df['Product_Name'])
p_name

In [ ]:
def product_recommend(Product_Name, cos_sim = similarity_matrix):
  recommended_prod = []
  idx = p_name[p_name == Product_Name].index[0]
  score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
  top_5_indexes = list(score_series.iloc[1:6].index)
  for i in top_5_indexes:
    recommended_prod.append(p_name.iloc[i])
  return recommended_prod

In [ ]:
product_recommend('DOVE BOTANIC BREAKAGE PROTECT SHAMPOO')

In [ ]:
df_sim_matrix = pd.DataFrame(similarity_matrix , index= p_name, columns= p_name)
df_sim_matrix

In [1]:
# textual cluster analysis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Product_Details'])

In [ ]:
true_k = 5
model = KMeans(n_clusters= true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
def print_cluster(i):
  print(f"Cluster{i}")
  for ind in order_centroids[i, :5]:
    print(f"{terms[ind]}")
  print

print("Top terms per cluster")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
  print_cluster(i)

In [ ]:
def show_recommend(product):
  Y = vectorizer.transform([product])
  prediction= model.predict(Y)

  print_cluster(prediction[0])

In [ ]:
show_recommend('olive oil')